In [1]:
import numpy as np
import random as rd
import math as mp
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
steps =  100
birds =  100
H     =    1
beta  =    0.5
ndim  =    3

init_vel_max = 18
init_vel_min = 10
init_radius = 50

In [86]:
pos=np.zeros((birds,ndim))
vel=np.zeros((birds,ndim))

In [87]:
rd.seed()

for i in range (birds):
    theta_x = rd.random() * mp.pi
    phi_x = rd.random() * 2 * pi
    theta_v = rd.random() * mp.pi
    phi_v = rd.random() * 2 * pi
    r = rd.random() * init_radius
    v = rd.random() * (init_vel_max - init_vel_min)
    
    pos[i][0]= r * mp.cos(phi_x) * mp.sin(theta_x)
    pos[i][1]= r * mp.sin(phi_x) * mp.sin(theta_x)
    pos[i][2]= r * mp.cos(theta_x)
    vel[i][0]= v * mp.cos(phi_v) * mp.sin(theta_v)
    vel[i][1]= v * mp.sin(phi_v) * mp.sin(theta_v)
    vel[i][2]= v * mp.cos(theta_v)

In [88]:
#Funzione cutoff per S0 e S1
def cutoff(x,d,e):
    if x <= d:
        return 1
    elif x < (d + e):
        return 0.5+0.5*np.tan((1/(x-d))+(1/(x-(d+e))))
    else:
        return 0

In [89]:
def Aij(xi,xj):
    return 1 / (pow(1+pow(np.linalg.norm(xi-xj),2),beta))

In [90]:
#Calcolo w per la velocità come da articolo
def w(x,y,v):
    w1=Y1/pow(1+pow(np.linalg.norm(x-y),2),B1)
    w2=cutoff(np.linalg.norm(v),D1,E1)
    w3=(1-w2)
    w4=(np.dot((y-x),v))/((np.linalg.norm(y-x)*np.linalg.norm(v))) 
    w5=(1-cutoff(w4,np.cos(S2),np.cos(S1) - np.cos(S2))) #correzione messo coseno al posto dell'angolo
    return w1*(w2+w3*w5)

In [91]:
f1 = open(f"vel3D birds={birds} steps={steps} b={B1}.txt", "w")
f2 = open(f"pos3D birds={birds} steps={steps} b={B1}.txt", "w")
f1.close()
f2.close()

for k in range (steps):
    
        N=1/(birds)
        N2=P1/(birds)
        N3=1/(birds)
        
        for i in range (birds):
            somma=0
            somma2=0
            somma3=0
            #A attrazione
            for j in range (birds):
                if i != j:
                    t1=(1-cutoff(np.linalg.norm(pos[i]-pos[j]),D0,E0))
                    t3=w(pos[i],pos[j],vel[i])
                    t4=(vel[j]-vel[i])
                    t = t1*t3*t4
                    somma=somma+t
            v1=N*somma
            
            #R repulsione
            for j in range (birds):
                if i != j:
                    d1=cutoff(np.linalg.norm(pos[i]-pos[j]),D0,E0)
                    d2=(pos[i]-pos[j])/(pow(1+pow(np.linalg.norm(pos[i]-pos[j]),2),B1))
                    d=d1*d2
                    somma2=somma2+d
            v2=N2*somma2 
        
            #B effetto bordo
            K=np.array([0,0,-1])
            for j in range (birds):
                if i != j:
                    d3=1/(1+pow(np.linalg.norm(pos[i]-pos[j]),2))
                    somma3=somma3+d3
            pv3=N3*somma3
            v3=C*cutoff(pv3,D3,E3)*np.cross(vel[i],K)
            if ndim==2:
                v3=np.delete(v3,2)
                
            #Attrito    
            v4 = max((alpha-beta*pow(np.linalg.norm(vel[i]),2)) * np.linalg.norm(vel[i]), -np.linalg.norm(vel[i])/friction_step) * vel[i] / np.linalg.norm(vel[i])
            
            vel[i]+= normalize(v1 + v2 + v3 + v4)*timestep
            pos[i]+= vel[i]*timestep
            
            vel3D = open(f"vel3D birds={birds} steps={steps} b={B1}.txt", "a+")
            pos3D = open(f"pos3D birds={birds} steps={steps} b={B1}.txt", "a+")
            vel3D.write(str(vel[i][0]) + " " + str(vel[i][1]) + " " + str(vel[i][2]) + "\n")
            pos3D.write(str(pos[i][0]) + " " + str(pos[i][1]) + " " + str(pos[i][2]) + "\n")
            vel3D.close()
            pos3D.close()

        print(k)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [153]:
#Calcolo u = velocità singolo uccello - velocità media ad un certo step
velstep=vel3D[steps-1,:,:]
U=np.zeros((birds,ndim))
VelMedia=np.mean(velstep,axis=0)
for j in range (birds):
    U[j] = velstep[j] - VelMedia
        
#controllo che u sia uguale a zero
controll= np.round(np.sum(U,axis=0),2)
print(controll)

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [67]:
#Funzione delta correlazione
def delta(sigma):
    if sigma == 0:
        return 1
    else:
        return 0
    
#Funzione correlazione C(r) per uno step
def correlation(r):
    velox=U
    posix=pos3D[steps-1,:,:]
    som=0
    somDelta=0
    for i in range (birds):
        for j in range (birds):
            if i != j:
                modvi=np.linalg.norm(velox[i])
                modvj=np.linalg.norm(velox[j])
                si=velox[i]/modvi
                sj=velox[j]/modvj
                r1=np.linalg.norm(posix[i]-posix[j])
                dist = np.round((r-r1), 0)
                delt=delta(dist)
                
                
                
                somj=np.dot(si,sj)*delt
                som = som + somj
                somDelta = somDelta + delt
    if somDelta == 0:
        return 0
    else:
        return (som / somDelta)

In [68]:
#Calcolo valore correlazione per ogni r dell'array 
massimo = 35
r2 = np.arange(0,massimo,1)
cor = []
r = []
for i in range (r2.size):
    result = np.round(correlation(r2[i]),2)
    if result != 0:
        cor.append(result)
        r.append(r2[i])
    print(i)
cor=np.array(cor)
r=np.array(r)

plt.plot(r, cor, marker = "o", color = 'red')
plt.title("Grafico correlazione")
plt.xlabel("R") 
plt.ylabel("Correlation(R)")
plt.show()

NameError: name 'U' is not defined

In [69]:
#Rappresentazione volume ad un certo step t
pts=pos3D[steps-1,:,:]
hull = ConvexHull(pts)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# Plot defining corner points
ax.plot(pts.T[0], pts.T[1], pts.T[2], "ko")

# 12 = 2 * 6 faces are the simplices (2 simplices per square face)
for s in hull.simplices:
    s = np.append(s, s[0])  # Here we cycle back to the first coordinate
    ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], "r-")

# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

plt.show()

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [70]:
vel3D[steps-1]

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [154]:
summation=[0] * 3

for j in range (birds):
    summation+=vel[j] / np.linalg.norm(vel[j])

pol = 1/birds * np.linalg.norm(summation)
print(pol)

0.9998929777417366
